In [12]:
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print("nah")

nah


In [13]:
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print("nah")

nah


In [ ]:
import pandas as pd
from scipy.stats import spearmanr
df = pd.read_csv("test.csv")

# get spearmans correlation bewteen Predicted_Score and Validation Score columns
rho, pval = spearmanr(df["Predicted_Score"], df["Validation Score"])
print(f"Spearman ρ = {rho:.4f} (p={pval:.4g}) on {len(df)} samples")


KeyError: 'Final Score'

In [77]:
MAIN_PROMPT = """
You are the Orchestration Layer of a Filipino-English LLM Judge system.

Your job is to decide whether the given information (in JSON format) is enough to produce a final judgment on translation quality.
{json}

You must follow these decision rules:
1. The fields "english_sentence" and "filipino_sentence" are absolutely essential. Do NOT proceed without them.
2. Check if the system has already asked for the **context of the sentence** (in "user_notes").
   - If YES, mark as complete.  
   - If NO, add a step to ask the user for it.
   Do not force the user to provide it.
3. Check if the system has already asked for any **additional information** needed for judging (in "user_notes").  
   - If YES, mark as complete.  
   - If NO, add a step to ask the user for it.
   Do not force the user to provide it.
4. Determine if the system needs to use any tools to proceed. If YES, add those tool steps.  
   If NO, list all tools that could still be used and their intended queries.

Available tools and their command formats:
- **Sentence similarity**: `similarity "english phrase/sentence" "filipino phrase/sentence"`
- **English Dictionary lookup**: `lookup_english "word"`
- **Filipino Dictionary lookup**: `lookup_filipino "word"`

**Your output must be ONLY a JSON object in the following format:**
{{
  "ready_to_decide": true/false,
  "reason": "Why you can or cannot decide yet",
  "steps": [
    "ask \"...\"",
    "similarity \"...\" \"...\"",
    "lookup_english \"...\""
    "lookup_filipino \"...\""
  ]
}}

Where:
- `"ready_to_decide"` is TRUE if all required info is present, otherwise FALSE.
- `"steps"` is the ordered list of actions the system must take next.

Return ONLY valid JSON. No extra text.
"""

In [79]:
import os
import json
import time
import argparse
import pandas as pd
import statistics
from scipy.stats import spearmanr
from tqdm import tqdm
import google.generativeai as genai
import numpy as np
import traceback

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Please set GOOGLE_API_KEY in your environment.")
genai.configure(api_key=GOOGLE_API_KEY)
MODEL_NAME = "gemini-2.5-flash-lite"

current_requests = 0
rate_limit_num = 5
rate_limit_wait = 1

def call_gemini(prompt: str):
    global current_requests
    global rate_limit_num
    global rate_limit_wait
    if current_requests >= rate_limit_num:
            print()
            print(f"Rate limit exceeded: {current_requests} > {rate_limit_num}")
            time.sleep(rate_limit_wait)
            current_requests = 0

    try:
        model = genai.GenerativeModel(MODEL_NAME)
        resp = model.generate_content(prompt)
        return resp.text.strip()
    except Exception as e:
        raise RuntimeError(f"Gemini call failed: {e}")

In [122]:
import json

class TranslationJudgeMemory:
    def __init__(self):
        self.memory = {
            "english_sentence": None,
            "filipino_sentence": None,
            "accuracy_weight": 1.5,
            "fluency_weight": 1.5,
            "completeness_weight": 0.5,
            "cultural_appropriateness_weight": 0.5,
            "user_notes": [],
            "tool_notes": []
        }

    def update(self, key, value):
        """Update a memory field."""
        if key in self.memory:
            self.memory[key] = value
        else:
            raise KeyError(f"Memory key '{key}' not found.")

    def add_note(self, note, source="user"):
        """Add note to either user_notes or tool_notes."""
        if source == "user":
            self.memory["user_notes"].append(note)
        elif source == "tool":
            self.memory["tool_notes"].append(note)

    def is_complete(self):
        """Check if we have enough info to proceed to scoring."""
        return all(self.memory["weights"])  # simple completion check

    def __str__(self):
        return json.dumps(self.memory, ensure_ascii=False, indent=2)


test = TranslationJudgeMemory()
# test.update("english_sentence", "The cat is on the roof.")
#test.update("filipino_sentence", "Ang pusa ay nasa bubong.")
#test.add_note("There is no context to this sentence. Just a simple statement about a cat on a roof.", source="user")

print(test)

{
  "english_sentence": null,
  "filipino_sentence": null,
  "accuracy_weight": 1.5,
  "fluency_weight": 1.5,
  "completeness_weight": 0.5,
  "cultural_appropriateness_weight": 0.5,
  "user_notes": [],
  "tool_notes": []
}


In [109]:
prompt = MAIN_PROMPT.format(json=test.__str__())
print(prompt)


You are the Orchestration Layer of a Filipino-English LLM Judge system.

Your job is to decide whether the given information (in JSON format) is enough to produce a final judgment on translation quality.
{
  "english_sentence": "The cat is on the roof.",
  "filipino_sentence": "Ang pusa ay nasa bubong.",
  "accuracy_weight": 1.5,
  "fluency_weight": 1.5,
  "completeness_weight": 0.5,
  "cultural_appropriateness_weight": 0.5,
  "user_notes": [
    "There is no context to this sentence. Just a simple statement about a cat on a roof."
  ],
  "tool_notes": []
}

You must follow these decision rules:
1. The fields "english_sentence" and "filipino_sentence" are absolutely essential. Do NOT proceed without them.
2. Check if the system has already asked for the **context of the sentence** (in "user_notes").
   - If YES, mark as complete.  
   - If NO, add a step to ask the user for it.
   Do not force the user to provide it.
3. Check if the system has already asked for any **additional inform

In [111]:
res = call_gemini(prompt)
print(res)

```json
{
  "ready_to_decide": false,
  "reason": "Information is not sufficient to produce a final judgment. The system needs to ask for context and any additional information required for judging.",
  "steps": [
    "ask \"Please provide the context for this sentence, if any.\"",
    "ask \"Please provide any additional information needed for judging.\"",
    "similarity \"The cat is on the roof.\" \"Ang pusa ay nasa bubong.\"",
    "lookup_english \"cat\"",
    "lookup_english \"roof\"",
    "lookup_filipino \"pusa\"",
    "lookup_filipino \"bubong\""
  ]
}
```


In [112]:
def parse_json_response(text: str):
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1:
        raise ValueError("No JSON object found.")
    obj = json.loads(text[start:end+1])
    return obj

In [113]:
res = parse_json_response(res)


In [114]:
res

{'ready_to_decide': False,
 'reason': 'Information is not sufficient to produce a final judgment. The system needs to ask for context and any additional information required for judging.',
 'steps': ['ask "Please provide the context for this sentence, if any."',
  'ask "Please provide any additional information needed for judging."',
  'similarity "The cat is on the roof." "Ang pusa ay nasa bubong."',
  'lookup_english "cat"',
  'lookup_english "roof"',
  'lookup_filipino "pusa"',
  'lookup_filipino "bubong"']}

In [115]:
for step in res['steps']:
    if step.startswith("ask"):
        question = step.split("\"")[1]
        print("I need to ask the user for: ", question)
    elif step.startswith("similarity"):
        phrases = step.split("\"")[1::2]
        print(f"Need to check similarity between: {phrases}")
    elif step.startswith("lookup_english"):
        word = step.split("\"")[1]
        print(f"Need to look up English word: {word}")
    elif step.startswith("lookup_filipino"):
        word = step.split("\"")[1]
        print(f"Need to look up Filipino word: {word}")

I need to ask the user for:  Please provide the context for this sentence, if any.
I need to ask the user for:  Please provide any additional information needed for judging.
Need to check similarity between: ['The cat is on the roof.', 'Ang pusa ay nasa bubong.']
Need to look up English word: cat
Need to look up English word: roof
Need to look up Filipino word: pusa
Need to look up Filipino word: bubong


In [128]:
UPDATE_PROMPT = """
You are the Orchestration Layer of a Filipino-English LLM Judge system.

Your job is to take in the user's input and extract the variables to be added to memory (in JSON format). The current memory is this:
{json}

Follow this format for adding to the memory and output lines based on what to add:
1. If you see that the user has supplied the English or Filipino text, then add:
english_sentence "(English sentence)"
filipino_sentence "(Filipino sentence)"
2. If the user has provided the context of the sentence, add it in the user notes:
note "(the context of the sentences)"
3. If the user has provided additional guidelines/instructions, then add it in the user notes:
note "(the additional instructions of the sentences)"
4. If the user has stated that no context or no additional instructions are to be provided, then add it in the user notes.

**Your output must be ONLY a JSON object in the following format:**
{{
  "updates": [
    "english_sentence \"The cat is on the roof.\"",
    "filipino_sentence \"Ang pusa ay nasa bubong.\"",
    "note \"There is no context to this sentence. Just a simple statement about a cat on a roof.\""
  ]
}}

The user has provided the following information:
{input}

Return ONLY valid JSON. No extra text.
"""

In [131]:
update = UPDATE_PROMPT.format(json=test.__str__(), input="English: The cat is on the roof. Filipino: Ang pusa ay nasa bubong. No additional context nor information.")
print(update)
res = call_gemini(update)
res = parse_json_response(res)
res


You are the Orchestration Layer of a Filipino-English LLM Judge system.

Your job is to take in the user's input and extract the variables to be added to memory (in JSON format). The current memory is this:
{
  "english_sentence": null,
  "filipino_sentence": null,
  "accuracy_weight": 1.5,
  "fluency_weight": 1.5,
  "completeness_weight": 0.5,
  "cultural_appropriateness_weight": 0.5,
  "user_notes": [],
  "tool_notes": []
}

Follow this format for adding to the memory and output lines based on what to add:
1. If you see that the user has supplied the English or Filipino text, then add:
english_sentence "(English sentence)"
filipino_sentence "(Filipino sentence)"
2. If the user has provided the context of the sentence, add it in the user notes:
note "(the context of the sentences)"
3. If the user has provided additional guidelines/instructions, then add it in the user notes:
note "(the additional instructions of the sentences)"
4. If the user has stated that no context or no addition

{'updates': ['english_sentence "The cat is on the roof."',
  'filipino_sentence "Ang pusa ay nasa bubong."',
  'note "No additional context nor information."']}

In [ ]:
import requests

def get_english_definition(word):
    url = f"https://api.dictionaryapi.dev/api/v2/entries/en/{word}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        meanings = []
        for meaning in data[0]["meanings"]:
            for definition in meaning["definitions"]:
                meanings.append(definition["definition"])
        return meanings
    else:
        return None

print(get_english_definition("suggest"))


['To imply but stop short of saying explicitly.', 'To make one suppose; cause one to suppose (something).', 'To mention something as an idea, typically in order to recommend it', 'To seduce; to prompt to evil; to tempt.']


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from huggingface_hub import hf_hub_download
import numpy as np

# Load LaBSE model
similarity_model_name = "sentence-transformers/LaBSE"
similarity_tokenizer = AutoTokenizer.from_pretrained(similarity_model_name)
similarity_model = AutoModel.from_pretrained(similarity_model_name)

def labse_similarity(text1: str, text2: str) -> float:
    # Tokenize
    inputs = similarity_tokenizer([text1, text2], return_tensors='pt', padding=True, truncation=True)

    with torch.no_grad():
        embeddings = similarity_model(**inputs).pooler_output  # shape: (2, hidden_size)

    # Normalize embeddings
    embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

    # Cosine similarity
    similarity = torch.matmul(embeddings[0], embeddings[1]).item()
    return similarity

score = labse_similarity("", "Kamusta ka naman")


Similarity: 0.2622


In [51]:
from wiktionaryparser import WiktionaryParser

# Initialize the parser once
parser = WiktionaryParser()
parser.set_default_language("english")  # Retrieve Tagalog definitions from English Wiktionary

def lookup_tagalog_word(word):
    """
    Looks up a word on English Wiktionary and returns Tagalog-related definitions if available.
    """
    entries = parser.fetch(word, "english")
    if not entries:
        return {"word": word, "found": False}

    results = []
    for entry in entries:
        # Filter definitions that may involve Tagalog, or keep all
        definitions = entry.get("definitions", [])
        if not definitions:
            continue
        for d in definitions:
            text = " ".join(d.get("text", []))
            pos = d.get("partOfSpeech")
            examples = d.get("examples", [])
            results.append({"pos": pos, "definition": text, "examples": examples})

    return {
        "word": word,
        "found": bool(results),
        "results": results
    }


In [133]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def get_tagalog_definition(word):
    # Configure headless Chrome
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=options)

    try:
        url = f"https://www.tagalog.com/dictionary/{word}"
        driver.get(url)

        # Give time for Cloudflare & JS rendering
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # First div with class "definition"
        first_def_div = soup.find("div", class_="definition")
        if not first_def_div:
            return None

        # Third div inside it
        inner_divs = first_def_div.find_all("div")
        if len(inner_divs) < 3:
            return None

        target_div = inner_divs[2]  # zero-indexed

        # Remove <span> tags
        for span in target_div.find_all("span"):
            span.unwrap()

        return target_div.get_text(strip=True)

    finally:
        driver.quit()

# Example usage
print(get_tagalog_definition("makikislap"))


None


In [135]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def get_tagalog_search_and_definitions(word):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=options)

    try:
        url = f"https://www.tagalog.com/#{word}"
        driver.get(url)

        time.sleep(3)  # wait for JS & Cloudflare

        soup = BeautifulSoup(driver.page_source, "html.parser")

        results = []

        for i in range(1, 4):
            result_id = f"search_result{i}"
            result_elem = soup.find(id=result_id)
            if not result_elem:
                continue

            # Clean search result inner HTML (unwrap spans)
            for span in result_elem.find_all("span"):
                span.unwrap()
            search_result_text = result_elem.get_text(strip=True)

            # First sibling div with definitions
            def_div = result_elem.find_next_sibling("div")
            if def_div:
                for span in def_div.find_all("span"):
                    span.unwrap()
                def_text = def_div.get_text(strip=True)
            else:
                def_text = None

            results.append({
                "search_result": search_result_text,
                "definition": def_text
            })

        return results

    finally:
        driver.quit()

# Example usage:
if __name__ == "__main__":
    results = get_tagalog_search_and_definitions("salamat")
    for idx, res in enumerate(results, 1):
        print(f"Search Result {idx}: {res['search_result']}")
        print(f"Definition {idx}: {res['definition']}\n")


Search Result 1: salamat
Definition 1: [expression]thank you•thanks18 Example Sentences

Search Result 2: matsala
Definition 2: [slang]thank you•* slang derived from the word "salamat"(root: salamat)

Search Result 3: lamat
Definition 3: [noun]crack•glassware

